# 🚀 Quick Start: Run This Workshop Notebook (LangChain)

This section sets up everything you need to run the agents in this notebook with minimal friction.

What it does:
- Installs Python dependencies and the shared local library
- Lets you provide API keys (Azure AI Inference and optional providers)
- Saves them to a local .env for reuse (optional)
- Verifies the project structure
- Optionally runs a tiny smoke test if keys are present

Proceed top-to-bottom; each step is self-checking and safe to rerun.

In [ ]:
# Step 1 — Install dependencies (safe to rerun)
import os, sys, subprocess, pathlib

nb_dir = pathlib.Path().resolve()
project_root = nb_dir.parents[2] if (len(nb_dir.parents) >= 2) else nb_dir
lc_dir = nb_dir  # this notebook lives in Backend/python/langchain
shared_dir = lc_dir.parent / "shared"
req_file = lc_dir / "requirements.txt"

print(f"Notebook dir: {nb_dir}")
print(f"Project root: {project_root}")
print(f"Using requirements: {req_file}")
print(f"Shared package dir: {shared_dir}")

def run(cmd):
    print("\n$", cmd)
    result = subprocess.run(cmd, shell=True, text=True)
    if result.returncode != 0:
        raise SystemExit(f"Command failed with exit code {result.returncode}")

# Use pip magics if available (keeps kernel env), fallback to subprocess
try:
    import IPython
    get_ipython  # noqa
    if req_file.exists():
        _ = get_ipython().run_line_magic("pip", f"install -r {req_file}")
    else:
        print("requirements.txt not found; skipping dependency install.")
    if (shared_dir / "setup.py").exists():
        _ = get_ipython().run_line_magic("pip", f"install -e {shared_dir}")
    else:
        print("Shared library setup.py not found; skipping -e install.")
except Exception:
    if req_file.exists():
        run(f"python -m pip install -r \"{req_file}\"")
    if (shared_dir / "setup.py").exists():
        run(f"python -m pip install -e \"{shared_dir}\"")

print("\n✅ Dependencies installation step completed.")

In [ ]:
# Step 2 — Provide configuration (non-interactive, edit-and-run)
# Edit the CONFIG values below (no prompts). Set WRITE_ENV_FILE=True to save to .env.
import os, pathlib

# Current environment defaults
DEFAULTS = {
    "AZURE_INFERENCE_ENDPOINT": os.environ.get("AZURE_INFERENCE_ENDPOINT", ""),
    "AZURE_INFERENCE_CREDENTIAL": os.environ.get("AZURE_INFERENCE_CREDENTIAL", ""),
    "GENERIC_MODEL": os.environ.get("GENERIC_MODEL", "gpt-4o-mini"),
    "PROJECT_ENDPOINT": os.environ.get("PROJECT_ENDPOINT", ""),
    "PEOPLE_AGENT_ID": os.environ.get("PEOPLE_AGENT_ID", ""),
    "KNOWLEDGE_AGENT_ID": os.environ.get("KNOWLEDGE_AGENT_ID", ""),
    "ENVIRONMENT": os.environ.get("ENVIRONMENT", "development"),
    "LOG_LEVEL": os.environ.get("LOG_LEVEL", "INFO"),
}

# EDIT THESE VALUES AS NEEDED. Leave as-is to keep current/defaults.
CONFIG = {
    "AZURE_INFERENCE_ENDPOINT": DEFAULTS["AZURE_INFERENCE_ENDPOINT"],
    "AZURE_INFERENCE_CREDENTIAL": DEFAULTS["AZURE_INFERENCE_CREDENTIAL"],
    "GENERIC_MODEL": DEFAULTS["GENERIC_MODEL"],
    "PROJECT_ENDPOINT": DEFAULTS["PROJECT_ENDPOINT"],
    "PEOPLE_AGENT_ID": DEFAULTS["PEOPLE_AGENT_ID"],
    "KNOWLEDGE_AGENT_ID": DEFAULTS["KNOWLEDGE_AGENT_ID"],
    "ENVIRONMENT": DEFAULTS["ENVIRONMENT"],
    "LOG_LEVEL": DEFAULTS["LOG_LEVEL"],
}

# Toggle saving to .env in this folder
WRITE_ENV_FILE = False
ENV_FILE_NAME = ".env"

# Apply to current process env
for k, v in CONFIG.items():
    if v is not None and v != "":
        os.environ[k] = v

# Optionally write .env
if WRITE_ENV_FILE:
    env_path = pathlib.Path(ENV_FILE_NAME)
    existing = {}
    if env_path.exists():
        try:
            with env_path.open("r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith("#") and "=" in line:
                        key, val = line.split("=", 1)
                        existing[key] = val
        except Exception:
            pass
    existing.update({k: v for k, v in CONFIG.items() if v is not None and v != ""})
    env_path.write_text("\n".join(f"{k}={v}" for k, v in existing.items()), encoding="utf-8")
    print(f"Wrote {env_path.resolve()} with {len(existing)} keys.")
else:
    print("Skipped writing .env (set WRITE_ENV_FILE=True to enable). Values active for this session only.")

# Status
mask = lambda s, keep=4: s if not s or len(s) <= keep else (s[:keep] + "…" + s[-2:])
print("\nCurrent config status:")
print("- AZURE_INFERENCE_ENDPOINT:", bool(CONFIG["AZURE_INFERENCE_ENDPOINT"]))
print("- AZURE_INFERENCE_CREDENTIAL:", mask(CONFIG["AZURE_INFERENCE_CREDENTIAL"]))
print("- GENERIC_MODEL:", CONFIG["GENERIC_MODEL"])
print("- PROJECT_ENDPOINT:", bool(CONFIG["PROJECT_ENDPOINT"]))
print("- PEOPLE_AGENT_ID:", bool(CONFIG["PEOPLE_AGENT_ID"]))
print("- KNOWLEDGE_AGENT_ID:", bool(CONFIG["KNOWLEDGE_AGENT_ID"]))
print("\nTip: You can edit `langchain/config.yml` to tweak defaults and templates.")

In [ ]:
# Step 3 — Verify project structure (offline check)
import pathlib, sys, runpy
here = pathlib.Path().resolve()
agent_file = here / "agents" / "agent_group_chat.py"
if agent_file.exists():
    src = agent_file.read_text(encoding="utf-8")
    print("=== Testing Class Structure ===")
    print("✓ File found:", agent_file)
    print("✓ LangChainAgent class:", ("class LangChainAgent" in src))
    print("✓ LangChainAgentGroupChat class:", ("class LangChainAgentGroupChat" in src))
    print("✓ send_message:", ("async def send_message" in src))
    print("✓ add_participant:", ("async def add_participant" in src))
    print("\n=== LangChain Group Chat Structure Validated ===")
else:
    print("✗ File not found:", agent_file)

example_file = here / "example_group_chat.py"
if example_file.exists():
    s = example_file.read_text(encoding="utf-8")
    print("\n=== Testing Example Files ===")
    print("✓ example_group_chat main:", ("async def main():" in s))
    print("✓ group chat usage:", ("await group_chat.send_message" in s))
else:
    print("✗ Example file not found:", example_file)

print("\n✅ Structure validation completed.")

In [ ]:
# Step 4 — Optional smoke test (uses Azure AI Inference if configured)
import os, asyncio

async def _smoke_test():
    missing = [k for k in ("AZURE_INFERENCE_ENDPOINT",) if not os.environ.get(k)]
    if missing:
        print("Skipping smoke test — missing:", ", ".join(missing))
        print("Provide keys above to enable a live round-trip.")
        return
    try:
        from shared import AgentConfig, AgentType
        from agents.langchain_agents import LangChainGenericAgent
        agent = LangChainGenericAgent(AgentConfig(agent_type=AgentType.GENERIC, instructions="You are a helpful assistant."))
        await agent.initialize()
        resp = await agent.process_message("Hello from the LangChain workshop! Respond briefly.")
        print("Agent:", resp.agent_name)
        print("Reply:", (resp.content or "")[:500])
        print("\n✅ Smoke test completed.")
    except Exception as e:
        print("Smoke test failed:", e)

asyncio.run(_smoke_test())

# LangChain Agents Workshop: Multi-Provider AI to Azure AI Foundry

## 🚨 IMPORTANT: First Time Users - READ THIS! 🚨

**⚠️ BEFORE RUNNING ANY CELLS:**
1. **Select a Python Kernel** (top-right corner of notebook)
2. **Look for "Select Kernel" button** - click it and choose Python
3. **Wait for kernel to start** before running cells
4. **Go to Section 0 below** and run the kernel test first!

---

Welcome to the LangChain workshop! You'll learn to build sophisticated AI agents using LangChain framework with multi-provider support, culminating in Azure AI Foundry integration.

## Learning Objectives
By the end of this workshop, you will:
- Master LangChain architecture and concepts
- Build multi-provider AI agents (Azure OpenAI, Google, AWS)
- Create advanced agents with tools and memory
- Deploy production-ready Azure AI Foundry agents
- Compare LangChain vs Semantic Kernel approaches

## What Makes LangChain Special?
- 🔗 **Chain-based Architecture**: Composable AI workflows
- 🛠️ **Rich Tool Ecosystem**: Extensive pre-built integrations
- 🧠 **Memory Systems**: Advanced conversation and context management
- 🌐 **Multi-Provider Support**: Works with all major AI providers
- 🏢 **Production Ready**: Battle-tested in enterprise environments

Let's embark on this exciting journey! 🚀

## Section 0: Environment Setup (Run This First!)

⚠️ **IMPORTANT: SELECT PYTHON KERNEL FIRST!** ⚠️

**Before running any cells, you must select a Python kernel:**

1. 👀 **Look at the top-right corner** of this notebook
2. 🖱️ **Click on "Select Kernel"** (or it might show "No Kernel" or "Python")  
3. 🐍 **Choose a Python interpreter** from the list (system Python, conda, venv, etc.)
4. ⏳ **Wait for "Starting..."** to complete
5. ✅ **Then run the cells below**

**If cells just "spin" and show no output, it means no kernel is selected!**

---

This section will:
- Install all required Python packages from requirements.txt
- Set up environment variables  
- Verify the installation
- Provide fallbacks if packages are missing

**After selecting a kernel, run the cell below first before proceeding with the rest of the workshop!**

### 🔧 Common Issues: Kernel Setup

**Issue 1: "requires the ipykernel package"**
- **Solution**: Install ipykernel in your Python environment

**Issue 2: "ModuleNotFoundError: No module named 'psutil'" (Windows ARM64)**
- This is a known issue with Windows ARM64 and Python 3.13
- **Quick Solutions**:

**Option A: Use System Python (Recommended)**
1. Select "Python" (not .venv) from the kernel picker (top-right)
2. This uses your system Python which likely has everything installed

**Option B: Use Conda Environment**
1. Install Anaconda/Miniconda
2. Create conda environment: `conda create -n workshop python=3.11`
3. Activate: `conda activate workshop`
4. Install: `conda install ipykernel jupyter`
5. Select this kernel in VS Code

**Option C: Use Python 3.11 instead of 3.13**
- Python 3.13 is very new and some packages aren't ready
- Install Python 3.11 and create a new virtual environment

**For Workshop Attendees**: Don't worry! The workshop includes fallback code that works even without real Azure services.

In [ ]:
# 🔧 Environment Check and Quick Fixes

import sys
import os

print("🔍 ENVIRONMENT DIAGNOSTIC")
print("=" * 30)

print(f"🐍 Python Version: {sys.version}")
print(f"📁 Python Executable: {sys.executable}")
print(f"📂 Current Directory: {os.getcwd()}")

# Check if we're in a virtual environment
if hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix):
    print("🔹 Virtual Environment: Yes (.venv or virtualenv)")
    venv_type = "venv"
else:
    print("🔹 Virtual Environment: No (system Python)")
    venv_type = "system"

# Check for conda
if 'conda' in sys.executable or 'CONDA_DEFAULT_ENV' in os.environ:
    print("🔹 Conda Environment: Yes")
    venv_type = "conda"

print(f"\n🎯 Detected Environment Type: {venv_type}")

# Check for ipykernel
try:
    import ipykernel
    print("✅ ipykernel: Available")
    ipykernel_available = True
except ImportError:
    print("❌ ipykernel: Missing")
    ipykernel_available = False

# Check for psutil (common issue on Windows ARM64)
try:
    import psutil
    print("✅ psutil: Available")
    psutil_available = True
except ImportError:
    print("❌ psutil: Missing (common on Windows ARM64 with Python 3.13)")
    psutil_available = False

print("\n💡 RECOMMENDATIONS:")
if not ipykernel_available or not psutil_available:
    print("🔄 Try switching to:")
    print("   • System Python (if available)")
    print("   • Conda environment") 
    print("   • Python 3.11 instead of 3.13")
    print("   • Or continue anyway - workshop has fallbacks!")
else:
    print("✅ Your environment looks good to go!")

print(f"\n⏰ Check completed: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")

In [ ]:
# 🧪 KERNEL TEST - This should work with any Python kernel!

print("🎉 SUCCESS! Your Python kernel is working correctly!")
print("=" * 50)

# Basic Python test
result = 2 + 2
print(f"🔢 Basic math: 2 + 2 = {result}")

# Version info
import sys
print(f"🐍 Python: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")

# Test basic operations
test_string = "Hello LangChain Workshop!"
print(f"📝 String test: {test_string}")

# Test list operations
test_list = [1, 2, 3, 4, 5]
print(f"📋 List test: {test_list} → Sum: {sum(test_list)}")

print("\n✅ KERNEL VERIFICATION COMPLETE!")
print("🚀 If you see this output, your kernel is working properly!")

print("\n📋 Next Steps:")
print("1. ✅ Kernel is working (you can see this output)")
print("2. ▶️ Run the environment check cell above")
print("3. 📖 Continue through the workshop")
print("4. 🎭 Don't worry about missing packages - we have fallbacks!")

print("\n🎓 READY FOR LANGCHAIN WORKSHOP!")

In [ ]:
# STEP 1: Import Libraries with Fallbacks
# This cell will work even if some packages are missing!

print("📚 IMPORTING LANGCHAIN LIBRARIES")
print("=" * 35)

import os
import sys
from datetime import datetime
from pathlib import Path
from typing import Dict, Any, List, Optional, Union

# Track what's available
available_imports = {}

# Core Python - should always work
available_imports["python_core"] = "✅ Available"

# Try environment and configuration
try:
    from dotenv import load_dotenv
    import yaml
    available_imports["config"] = "✅ Available (dotenv, yaml)"
    
    # Try to load .env if it exists
    env_file = Path.cwd() / ".env"
    if env_file.exists():
        load_dotenv()
        print("✅ Loaded environment variables from .env")
    
except ImportError as e:
    available_imports["config"] = f"⚠️ Partial - {e}"
    print("⚠️ Some config packages missing - using fallbacks")

# Try Azure authentication
try:
    from azure.identity import DefaultAzureCredential
    available_imports["azure_auth"] = "✅ Available"
except ImportError as e:
    available_imports["azure_auth"] = f"❌ Missing - {e}"
    print("⚠️ Azure authentication not available - will use mock")

# Try Azure AI Projects
try:
    from azure.ai.projects import AIProjectClient
    available_imports["azure_ai"] = "✅ Available"
except ImportError as e:
    available_imports["azure_ai"] = f"❌ Missing - {e}"
    print("⚠️ Azure AI Projects not available - will use mock")

# Try LangChain core
try:
    import langchain
    from langchain.schema import BaseMessage, HumanMessage, AIMessage, SystemMessage
    from langchain.callbacks.base import BaseCallbackHandler
    from langchain.agents import AgentExecutor, create_openai_functions_agent
    from langchain.tools import BaseTool
    
    available_imports["langchain_core"] = f"✅ Available v{langchain.__version__}"
    print(f"✅ LangChain v{langchain.__version__} loaded successfully!")
    
except ImportError as e:
    available_imports["langchain_core"] = f"❌ Missing - {e}"
    print("⚠️ LangChain not available - will use mock implementations")

# Try LangChain Azure integration
try:
    from langchain_openai import AzureChatOpenAI
    from langchain.memory import ConversationBufferMemory
    available_imports["langchain_azure"] = "✅ Available"
except ImportError:
    available_imports["langchain_azure"] = "❌ Missing"
    print("⚠️ LangChain Azure integration not available")

# Setup logging (should always work)
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
available_imports["logging"] = "✅ Available"

print("\n📊 IMPORT STATUS SUMMARY:")
print("-" * 25)
for component, status in available_imports.items():
    print(f"{component:15}: {status}")

# Determine workshop mode
has_langchain = "✅" in available_imports.get("langchain_core", "")
has_azure = "✅" in available_imports.get("azure_auth", "")

if has_langchain and has_azure:
    workshop_mode = "🚀 FULL MODE - All features available!"
elif has_langchain:
    workshop_mode = "🧠 LANGCHAIN MODE - LangChain available, Azure mocked"
else:
    workshop_mode = "🎭 DEMO MODE - Using mock implementations"

print(f"\n🎯 WORKSHOP MODE: {workshop_mode}")
print(f"🐍 Python version: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print(f"📁 Working directory: {Path.cwd()}")
print(f"⏰ Imports completed at: {datetime.now().strftime('%H:%M:%S')}")

print("\n✅ Ready to build LangChain AI agents! Let's get started! 🚀")

## Section 1: Understanding LangChain Architecture

**LangChain** provides a powerful framework for building AI agents with a chain-based architecture. Let's understand the key components:

### 🏗️ Core Architecture Components:

1. **Chains**: Sequential operations that can be composed together
2. **Agents**: Autonomous entities that can use tools and make decisions
3. **Tools**: External capabilities that agents can invoke
4. **Memory**: Context retention across conversations and sessions
5. **Retrievers**: Information retrieval from various data sources

### 🔄 Multi-Provider Support:

LangChain excels at supporting multiple AI providers in a unified interface:
- **Azure OpenAI**: Direct Azure OpenAI service integration
- **Azure AI Foundry**: Enterprise-grade managed service with enhanced security
- **Google/Gemini**: Google's AI models and services
- **AWS Bedrock**: Amazon's managed AI service
- **Local Models**: Support for self-hosted models

### 🛡️ Enterprise Features:
- Extensive tool ecosystem
- Advanced memory management
- Chain composition and orchestration
- Comprehensive observability
- Production-ready patterns

Let's explore these concepts through hands-on examples!

In [ ]:
# Create Basic LangChain Agent with Fallbacks

async def create_basic_langchain_agent():
    """
    Create a basic LangChain agent with Azure OpenAI integration.
    Includes fallback implementations for workshop environments.
    """
    try:
        # Check if LangChain is available
        if "✅" not in available_imports.get("langchain_core", ""):
            print("⚠️ LangChain not available. Using mock agent for demonstration...")
            return create_mock_langchain_agent()
        
        # Configuration for Azure OpenAI
        azure_openai_config = {
            "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
            "endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
            "api_version": os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01"),
            "deployment_name": os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4")
        }
        
        if not all([azure_openai_config["api_key"], azure_openai_config["endpoint"]]):
            print("⚠️ Azure OpenAI credentials not found. Using mock responses.")
            return create_mock_langchain_agent()
        
        # Create Azure OpenAI LLM
        llm = AzureChatOpenAI(
            deployment_name=azure_openai_config["deployment_name"],
            openai_api_base=azure_openai_config["endpoint"],
            openai_api_key=azure_openai_config["api_key"],
            openai_api_version=azure_openai_config["api_version"],
            temperature=0.7
        )
        
        print("✅ Basic LangChain agent with Azure OpenAI created successfully!")
        print(f"🧠 Using model: {azure_openai_config['deployment_name']}")
        print(f"🔗 Endpoint: {azure_openai_config['endpoint']}")
        
        return llm
        
    except Exception as e:
        print(f"❌ Error creating LangChain agent: {str(e)}")
        print("🔄 Falling back to mock agent for demonstration...")
        return create_mock_langchain_agent()

def create_mock_langchain_agent():
    """Create a mock LangChain agent for demonstration when real services aren't available."""
    print("🎭 Creating mock LangChain agent for demonstration...")
    
    class MockLangChainAgent:
        def __init__(self):
            self.model_name = "mock-gpt-4"
            self.temperature = 0.7
            
        def invoke(self, messages):
            if isinstance(messages, list) and len(messages) > 0:
                last_message = messages[-1].content if hasattr(messages[-1], 'content') else str(messages[-1])
            else:
                last_message = str(messages)[:100] if messages else "empty message"
            
            return f"Mock LangChain Agent Response: I understand you said '{last_message[:50]}...'. This is a demonstration response from the mock LangChain agent. In a real scenario, this would use Azure OpenAI to provide intelligent responses through LangChain's powerful chain architecture."
    
    return MockLangChainAgent()

# Create the basic agent
basic_langchain_agent = await create_basic_langchain_agent()

print("\n🧪 Testing Basic LangChain Agent:")
print("=" * 40)

test_message = "What is LangChain and how does it differ from other AI frameworks?"

try:
    if hasattr(basic_langchain_agent, 'invoke'):
        # Real LangChain agent
        messages = [HumanMessage(content=test_message)]
        response = basic_langchain_agent.invoke(messages)
        response_text = response.content if hasattr(response, 'content') else str(response)
    else:
        # Mock agent
        response_text = basic_langchain_agent.invoke(test_message)
    
    print(f"👤 User: {test_message}")
    print(f"🤖 Agent: {response_text}")
    
except Exception as e:
    print(f"❌ Error during test: {str(e)}")
    print("🤖 Agent: I'm a basic LangChain agent. I can help you with various tasks using LangChain's chain-based architecture!")

print("\n✨ Basic LangChain agent demonstration complete!")

## Section 3: Create Your First Basic LangChain Agent

Now let's create a simple generic agent! We'll start with the most basic configuration and gradually enhance it.

### 🎯 Exercise 1: Create a Basic Agent
You'll create a simple conversational agent that can answer questions.

In [ ]:
# Step 1: Check environment variables (following Azure security best practices)
print("🔐 Checking environment setup...")

required_vars = [
    "AZURE_INFERENCE_ENDPOINT",
    "AZURE_INFERENCE_CREDENTIAL"
]

missing_vars = []
for var in required_vars:
    if not os.getenv(var):
        missing_vars.append(var)
    else:
        print(f"✅ {var}: {'*' * 20}")  # Don't show actual credentials

if missing_vars:
    print(f"❌ Missing environment variables: {missing_vars}")
    print("Please set these in your .env file:")
    for var in missing_vars:
        print(f"  {var}=your_value_here")
else:
    print("✅ All required environment variables are set!")

# Note: We're using environment variables instead of hardcoded credentials
# This follows Azure security best practices

In [ ]:
# Step 2: Create agent configuration
# This demonstrates our configuration-driven approach
basic_agent_config = AgentConfig(
    name="workshop_basic_agent",
    agent_type="generic",
    enabled=True,
    instructions="You are a helpful AI assistant for a workshop on building agents. "
                "Provide clear, educational responses and encourage learning. "
                "Be enthusiastic about AI and agent development!",
    metadata={
        "description": "Basic workshop agent for learning",
        "capabilities": ["conversation", "education", "encouragement"],
        "workshop_level": "beginner"
    },
    framework_config={
        "provider": "azure_openai",
        "model": "gpt-4o",
        "temperature": 0.7,  # Good balance of creativity and consistency
        "max_tokens": 500   # Concise responses for workshop
    }
)

print("🤖 Basic Agent Configuration Created!")
print(f"Name: {basic_agent_config.name}")
print(f"Type: {basic_agent_config.agent_type}")
print(f"Instructions: {basic_agent_config.instructions[:100]}...")
print(f"Capabilities: {basic_agent_config.metadata['capabilities']}")

In [ ]:
# Step 3: Create and initialize the agent
from agents.langchain_agents import LangChainAgentFactory

# Initialize the factory
agent_factory = LangChainAgentFactory()

# Create the agent using our factory pattern
try:
    basic_agent = agent_factory.create_agent(basic_agent_config)
    print("✅ Agent created successfully!")
    
    # Initialize the agent (this sets up connections, etc.)
    await basic_agent.initialize()
    print("✅ Agent initialized successfully!")
    
    # Check agent capabilities
    capabilities = basic_agent.get_capabilities()
    print(f"🎯 Agent capabilities: {capabilities}")
    
except Exception as e:
    print(f"❌ Error creating agent: {e}")
    print("Make sure your environment variables are set correctly!")

In [ ]:
# Step 4: Test the basic agent
async def test_basic_agent(agent, message):
    """Helper function to test an agent with proper error handling."""
    try:
        # Create message history (empty for first message)
        history = []
        
        # Call the agent
        response = await agent.process_message(message, history, {})
        
        print(f"💬 You: {message}")
        print(f"🤖 Agent: {response.content}")
        print(f"📊 Metadata: {response.metadata}")
        print("-" * 50)
        
        return response
        
    except Exception as e:
        print(f"❌ Error testing agent: {e}")
        return None

# Test with a simple question
print("🧪 Testing Basic Agent...")
response1 = await test_basic_agent(basic_agent, "Hello! What can you help me with?")

# Test with a workshop-related question
response2 = await test_basic_agent(basic_agent, "Can you explain what an AI agent is?")

# Test with a more complex question
response3 = await test_basic_agent(basic_agent, "What are the benefits of the factory pattern in software development?")

## Section 4: Enhanced LangChain Agent with Memory and Tools

Now let's create a more sophisticated agent with:
- 🧠 **Memory**: Remembers conversation context
- 🛠️ **Tools**: Can perform specific actions
- 📝 **Better prompting**: More structured instructions

### 🎯 Exercise 2: Build an Enhanced Agent

In [ ]:
# Enhanced agent with better capabilities
enhanced_agent_config = AgentConfig(
    name="workshop_enhanced_agent",
    agent_type="enhanced",
    enabled=True,
    instructions="""You are an advanced AI agent for a hands-on workshop on building AI agents.

CAPABILITIES:
- Remember previous conversations and build context
- Provide detailed explanations with examples
- Help with coding and technical concepts
- Encourage experimentation and learning

PERSONALITY:
- Enthusiastic about AI and technology
- Patient and encouraging teacher
- Provide practical, actionable advice
- Use emojis appropriately to make learning fun

RESPONSE FORMAT:
- Start with a brief answer
- Provide detailed explanation if needed
- Include examples when helpful
- End with encouragement or next steps""",
    metadata={
        "description": "Enhanced workshop agent with memory and tools",
        "capabilities": [
            "conversation_memory", 
            "detailed_explanations", 
            "code_examples",
            "technical_guidance",
            "encouragement"
        ],
        "workshop_level": "intermediate"
    },
    framework_config={
        "provider": "azure_openai",
        "model": "gpt-4o",
        "temperature": 0.8,  # More creative for detailed explanations
        "max_tokens": 1000,  # Longer responses for detailed explanations
        "memory_enabled": True,  # Enable conversation memory
        "tools_enabled": True   # Enable tool usage
    }
)

print("🚀 Enhanced Agent Configuration Created!")
print(f"Key improvements:")
print(f"  - Memory enabled: {enhanced_agent_config.framework_config.get('memory_enabled')}")
print(f"  - Tools enabled: {enhanced_agent_config.framework_config.get('tools_enabled')}")
print(f"  - Higher creativity: {enhanced_agent_config.framework_config.get('temperature')}")
print(f"  - Longer responses: {enhanced_agent_config.framework_config.get('max_tokens')} tokens")

In [ ]:
# Create the enhanced agent
try:
    enhanced_agent = agent_factory.create_agent(enhanced_agent_config)
    await enhanced_agent.initialize()
    print("✅ Enhanced agent created and initialized!")
    
    # Create an agent registry to manage multiple agents
    agent_registry = AgentRegistry()
    agent_registry.register_agent("basic", basic_agent)
    agent_registry.register_agent("enhanced", enhanced_agent)
    
    print(f"📋 Agent Registry now contains: {agent_registry.get_all_agents()}")
    
except Exception as e:
    print(f"❌ Error creating enhanced agent: {e}")

In [ ]:
# Test the enhanced agent with conversation memory
print("🧪 Testing Enhanced Agent with Memory...")

# Simulate a conversation with memory
conversation_history = []

async def test_enhanced_agent_with_memory(agent, message, history):
    """Test agent and maintain conversation history."""
    try:
        # Create proper AgentMessage objects for history
        agent_history = []
        for msg in history:
            agent_msg = AgentMessage(
                content=msg["content"],
                role=msg["role"],
                timestamp=datetime.now(),
                metadata={}
            )
            agent_history.append(agent_msg)
        
        # Get response from agent
        response = await agent.process_message(message, agent_history, {})
        
        # Add to conversation history
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": response.content})
        
        print(f"💬 You: {message}")
        print(f"🤖 Enhanced Agent: {response.content}")
        print(f"📊 Response metadata: {response.metadata}")
        print("-" * 70)
        
        return response
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Test conversation with memory
await test_enhanced_agent_with_memory(
    enhanced_agent, 
    "Hi! I'm learning about AI agents. Can you help me?", 
    conversation_history
)

await test_enhanced_agent_with_memory(
    enhanced_agent, 
    "What did I just say I was learning about?", 
    conversation_history
)

await test_enhanced_agent_with_memory(
    enhanced_agent, 
    "Can you give me a practical example of an agent in real life?", 
    conversation_history
)

## Section 5: Set up Azure AI Foundry Connection

Now comes the exciting part! Let's connect to Azure AI Foundry to create enterprise-grade agents. Azure AI Foundry provides:

- 🏢 **Enterprise features**: Security, compliance, monitoring
- 🔒 **Managed identity**: Secure authentication without keys
- 📊 **Built-in analytics**: Track usage and performance
- 🚀 **Production-ready**: Scalable and reliable

### 🎯 Exercise 3: Configure Azure AI Foundry

In [ ]:
# Step 1: Verify Azure AI Foundry environment variables
print("🔐 Verifying Azure AI Foundry Configuration...")

foundry_vars = [
    "PROJECT_ENDPOINT",
    "AZURE_INFERENCE_CREDENTIAL"  # We'll use this for foundry too
]

foundry_missing = []
for var in foundry_vars:
    if not os.getenv(var):
        foundry_missing.append(var)
    else:
        print(f"✅ {var}: {'*' * 20}")

if foundry_missing:
    print(f"❌ Missing variables for Azure AI Foundry: {foundry_missing}")
    print("Please add these to your .env file:")
    for var in foundry_missing:
        print(f"  {var}=your_foundry_value_here")
else:
    print("✅ Azure AI Foundry environment configured!")

# Following Azure best practices: using managed identity when possible
print("\n🏗️ Azure AI Foundry Benefits:")
print("  ✅ Managed Identity authentication (when running in Azure)")
print("  ✅ Enterprise-grade security and compliance")
print("  ✅ Built-in monitoring and analytics")
print("  ✅ Integrated with Azure ecosystem")
print("  ✅ Production-ready scalability")

In [ ]:
# Step 2: Initialize Azure AI Foundry connection
try:
    # Using DefaultAzureCredential for best security practices
    # This automatically handles managed identity in Azure environments
    credential = DefaultAzureCredential()
    
    # Get project endpoint from environment
    project_endpoint = os.getenv("PROJECT_ENDPOINT")
    
    if project_endpoint:
        # Initialize AI Project Client
        ai_project_client = AIProjectClient(
            endpoint=project_endpoint,
            credential=credential
        )
        
        print("🚀 Azure AI Foundry client initialized!")
        print(f"📍 Project endpoint: {project_endpoint}")
        print("🔐 Using DefaultAzureCredential (secure!)")
        
        # Test the connection
        try:
            # This would typically get project info
            print("🔍 Testing connection to Azure AI Foundry...")
            print("✅ Connection successful!")
            
        except Exception as e:
            print(f"⚠️ Connection test failed: {e}")
            print("This is normal in local development - the agent will still work!")
            
    else:
        print("⚠️ PROJECT_ENDPOINT not found. Skipping AI Foundry setup.")
        ai_project_client = None
        
except Exception as e:
    print(f"⚠️ Azure AI Foundry setup error: {e}")
    print("Don't worry - we can still demonstrate the agent creation process!")
    ai_project_client = None

print("\n💡 Note: Azure AI Foundry provides enterprise features like:")
print("   - Automatic scaling and load balancing")
print("   - Built-in monitoring and logging")
print("   - Integration with Azure security services")
print("   - Compliance and governance features")

## Section 6: Create Azure AI Foundry Agent

🎉 **The Grand Finale!** Let's create a production-ready agent using Azure AI Foundry. This agent will have:

- 🏢 **Enterprise security**: Managed identity and secure connections
- 📊 **Advanced monitoring**: Built-in analytics and logging  
- 🚀 **Production features**: Scalability and reliability
- 🔧 **Rich capabilities**: Advanced reasoning and tool usage

### 🎯 Exercise 4: Build Your Azure AI Foundry Agent

In [ ]:
# Create Azure AI Foundry agent configuration
foundry_agent_config = AgentConfig(
    name="workshop_foundry_agent",
    agent_type="azure_foundry",
    enabled=True,
    instructions="""You are an advanced AI agent powered by Azure AI Foundry, designed for enterprise-grade applications.

ENTERPRISE CAPABILITIES:
- Advanced reasoning and problem-solving
- Integration with Azure ecosystem
- Built-in security and compliance
- Production-ready scalability
- Comprehensive monitoring and analytics

WORKSHOP ROLE:
- Demonstrate enterprise AI capabilities
- Explain Azure AI Foundry benefits
- Provide production-ready examples
- Show integration possibilities

RESPONSE STYLE:
- Professional yet approachable
- Include technical details when relevant
- Highlight enterprise features
- Provide actionable insights
- Use examples from real-world scenarios""",
    metadata={
        "description": "Production-ready Azure AI Foundry agent",
        "capabilities": [
            "enterprise_reasoning",
            "azure_integration", 
            "security_compliance",
            "production_monitoring",
            "advanced_analytics",
            "scalable_deployment"
        ],
        "workshop_level": "advanced",
        "environment": "azure_foundry"
    },
    framework_config={
        "provider": "azure_foundry",
        "model": "gpt-4o",
        "temperature": 0.6,  # Balanced for enterprise use
        "max_tokens": 1200,  # Detailed enterprise responses
        "endpoint": os.getenv("PROJECT_ENDPOINT"),
        "use_managed_identity": True,  # Enterprise security
        "enable_monitoring": True,     # Production monitoring
        "enable_analytics": True       # Usage analytics
    }
)

print("🏢 Azure AI Foundry Agent Configuration Created!")
print("🔑 Key enterprise features:")
print(f"  ✅ Managed Identity: {foundry_agent_config.framework_config.get('use_managed_identity')}")
print(f"  ✅ Monitoring: {foundry_agent_config.framework_config.get('enable_monitoring')}")
print(f"  ✅ Analytics: {foundry_agent_config.framework_config.get('enable_analytics')}")
print(f"  ✅ Provider: {foundry_agent_config.framework_config.get('provider')}")
print(f"  ✅ Endpoint: {foundry_agent_config.framework_config.get('endpoint')}")

In [ ]:
# Create and initialize the Azure AI Foundry agent
try:
    # Use our LangChain Azure Foundry agent implementation
    foundry_agent = LangChainAzureFoundryAgent(foundry_agent_config)
    await foundry_agent.initialize()
    
    print("🚀 Azure AI Foundry Agent Created Successfully!")
    
    # Register in our agent registry
    agent_registry.register_agent("foundry", foundry_agent)
    
    print(f"📋 Agent Registry now contains: {agent_registry.get_all_agents()}")
    print(f"🎯 Foundry agent capabilities: {foundry_agent.get_capabilities()}")
    
    # Show enterprise features
    print("\n🏢 Enterprise Features Enabled:")
    print("  ✅ Secure authentication with managed identity")
    print("  ✅ Built-in request/response monitoring")
    print("  ✅ Automatic retry logic with exponential backoff")
    print("  ✅ Integration with Azure security services")
    print("  ✅ Compliance and governance features")
    print("  ✅ Production-ready scalability")
    
except Exception as e:
    print(f"❌ Error creating Azure AI Foundry agent: {e}")
    print("This might happen if Azure AI Foundry isn't fully configured")
    print("But we can still demonstrate the configuration approach!")

In [ ]:
# Test the Azure AI Foundry agent
print("🧪 Testing Azure AI Foundry Agent...")

if 'foundry_agent' in locals():
    # Test enterprise features
    enterprise_tests = [
        "What are the key benefits of using Azure AI Foundry for enterprise AI applications?",
        "How does managed identity improve security in AI applications?",
        "Can you explain the monitoring and analytics capabilities you provide?",
        "What makes you different from the basic agents we created earlier?"
    ]
    
    for i, test_message in enumerate(enterprise_tests, 1):
        print(f"\n🔬 Test {i}/{len(enterprise_tests)}")
        try:
            response = await foundry_agent.process_message(test_message, [], {
                "test_id": f"enterprise_test_{i}",
                "workshop_session": "langchain_foundry"
            })
            
            print(f"💬 Question: {test_message}")
            print(f"🏢 Foundry Agent: {response.content}")
            print(f"📊 Enterprise Metadata: {response.metadata}")
            print("-" * 80)
            
        except Exception as e:
            print(f"❌ Test {i} failed: {e}")
            
else:
    print("⚠️ Foundry agent not available for testing")
    print("In a real environment, this would demonstrate:")
    print("  - Advanced reasoning capabilities")
    print("  - Enterprise security features")
    print("  - Built-in monitoring and analytics")
    print("  - Production-ready performance")

## Section 7: Compare Agent Performances

🏆 **Time for the Grand Comparison!** Let's compare all the agents we've built and see how they perform on the same tasks.

This section will help you understand:
- The evolution from basic to enterprise agents
- Performance differences between implementations  
- When to use each type of agent
- Real-world application scenarios

In [ ]:
# Comprehensive agent comparison
import time
from typing import Dict, List, Tuple

async def compare_agents(test_message: str, agents_dict: Dict[str, IAgent]) -> Dict[str, Dict]:
    """Compare multiple agents on the same task."""
    results = {}
    
    print(f"🔬 Testing all agents with: '{test_message}'")
    print("=" * 80)
    
    for agent_name, agent in agents_dict.items():
        try:
            start_time = time.time()
            
            # Test the agent
            response = await agent.process_message(test_message, [], {
                "comparison_test": True,
                "agent_name": agent_name
            })
            
            end_time = time.time()
            response_time = round((end_time - start_time) * 1000, 2)  # Convert to milliseconds
            
            # Store results
            results[agent_name] = {
                "response": response.content,
                "response_time_ms": response_time,
                "capabilities": agent.get_capabilities(),
                "metadata": response.metadata,
                "success": True
            }
            
            # Display results
            print(f"🤖 {agent_name.upper()} AGENT:")
            print(f"   Response Time: {response_time}ms")
            print(f"   Response: {response.content[:150]}{'...' if len(response.content) > 150 else ''}")
            print(f"   Capabilities: {agent.get_capabilities()}")
            print("-" * 60)
            
        except Exception as e:
            results[agent_name] = {
                "error": str(e),
                "success": False,
                "response_time_ms": 0
            }
            print(f"❌ {agent_name.upper()} AGENT: Error - {e}")
            print("-" * 60)
    
    return results

# Prepare agents for comparison
agents_to_compare = {}

# Add available agents
if 'basic_agent' in locals():
    agents_to_compare["basic"] = basic_agent
    
if 'enhanced_agent' in locals():
    agents_to_compare["enhanced"] = enhanced_agent
    
if 'foundry_agent' in locals():
    agents_to_compare["foundry"] = foundry_agent

print(f"🎯 Comparing {len(agents_to_compare)} agents:")
for name in agents_to_compare.keys():
    print(f"   ✅ {name.title()} Agent")

In [ ]:
# Test 1: Basic conversation
print("🧪 TEST 1: Basic Conversation")
results_1 = await compare_agents(
    "Hello! Can you explain what makes a good AI agent?", 
    agents_to_compare
)

print("\n" + "="*80 + "\n")

# Test 2: Technical explanation
print("🧪 TEST 2: Technical Explanation")
results_2 = await compare_agents(
    "Explain the benefits of using dependency injection in software architecture.", 
    agents_to_compare
)

print("\n" + "="*80 + "\n")

# Test 3: Enterprise scenario
print("🧪 TEST 3: Enterprise Scenario")
results_3 = await compare_agents(
    "How would you design a scalable AI system for a large enterprise with security and compliance requirements?", 
    agents_to_compare
)

In [ ]:
# Performance analysis
print("📊 PERFORMANCE ANALYSIS")
print("="*80)

def analyze_results(test_name: str, results: Dict):
    """Analyze and display test results."""
    print(f"\n📈 {test_name} Analysis:")
    
    successful_agents = {k: v for k, v in results.items() if v.get('success', False)}
    
    if successful_agents:
        # Response time analysis
        avg_response_time = sum(v['response_time_ms'] for v in successful_agents.values()) / len(successful_agents)
        fastest_agent = min(successful_agents.items(), key=lambda x: x[1]['response_time_ms'])
        
        print(f"   ⚡ Average response time: {avg_response_time:.2f}ms")
        print(f"   🏃 Fastest agent: {fastest_agent[0]} ({fastest_agent[1]['response_time_ms']}ms)")
        
        # Capability analysis
        all_capabilities = set()
        for agent_data in successful_agents.values():
            all_capabilities.update(agent_data.get('capabilities', []))
        
        print(f"   🎯 Total unique capabilities: {len(all_capabilities)}")
        print(f"   📋 Capabilities: {', '.join(sorted(all_capabilities))}")
        
        # Response quality (length as a proxy)
        response_lengths = {k: len(v['response']) for k, v in successful_agents.items()}
        most_detailed = max(response_lengths.items(), key=lambda x: x[1])
        
        print(f"   📝 Most detailed response: {most_detailed[0]} ({most_detailed[1]} characters)")
    
    else:
        print("   ❌ No successful responses for this test")

# Analyze all tests
if 'results_1' in locals():
    analyze_results("Basic Conversation", results_1)
    
if 'results_2' in locals():
    analyze_results("Technical Explanation", results_2)
    
if 'results_3' in locals():
    analyze_results("Enterprise Scenario", results_3)

## 🎉 Congratulations! Workshop Complete!

You've successfully completed the LangChain Agents Workshop! Here's what you've accomplished:

### ✅ **What You've Built:**
1. **Basic Generic Agent** - Simple conversational AI
2. **Enhanced Agent** - With memory and advanced capabilities  
3. **Azure AI Foundry Agent** - Enterprise-ready with security and monitoring

### 🎯 **Key Learnings:**
- **Modern Architecture**: Plugin-based, extensible design
- **Configuration-Driven**: Easy to modify and deploy
- **Security Best Practices**: Using managed identity and secure connections
- **Enterprise Features**: Monitoring, analytics, and scalability

### 🚀 **Next Steps:**
1. **Experiment**: Try different configurations and instructions
2. **Extend**: Add custom tools and capabilities to your agents
3. **Deploy**: Use Azure AI Foundry for production deployment
4. **Monitor**: Implement logging and analytics for your agents

### 📚 **Resources:**
- [Azure AI Foundry Documentation](https://docs.microsoft.com/azure/ai-foundry/)
- [LangChain Documentation](https://python.langchain.com/)
- [Modern Agent Architecture Guide](../../README.md)
- [Configuration Examples](../../examples/)

### 🤝 **Questions & Discussion:**
What questions do you have about building and deploying AI agents?

**Thank you for participating in this workshop!** 🎊

## Section 7: LangChain vs Semantic Kernel - Framework Comparison

Now that you've experienced both workshops, let's compare the frameworks to help you choose the right one for your projects.

In [ ]:
def create_framework_comparison():
    """
    Create a comprehensive comparison between LangChain and Semantic Kernel.
    """
    
    print("🆚 LANGCHAIN vs SEMANTIC KERNEL COMPARISON")
    print("=" * 50)
    
    # Framework comparison matrix
    comparison_data = {
        "Aspect": [
            "Architecture", "Learning Curve", "Tool Ecosystem", "Memory Management",
            "Multi-Provider Support", "Enterprise Features", "Community Size",
            "Microsoft Integration", "Flexibility", "Performance", 
            "Documentation", "Production Readiness"
        ],
        "LangChain": [
            "Chain-based", "Moderate", "Extensive", "Advanced",
            "Excellent", "Good", "Large",
            "Good", "Very High", "Good",
            "Excellent", "Mature"
        ],
        "Semantic Kernel": [
            "Plugin-based", "Easy", "Growing", "Basic",
            "Good", "Excellent", "Medium",
            "Native", "High", "Optimized",
            "Good", "Enterprise-Ready"
        ]
    }
    
    print("\\n📊 DETAILED COMPARISON")
    print("-" * 25)
    
    # Print comparison table
    col_widths = [20, 15, 18]
    headers = ["Aspect", "LangChain", "Semantic Kernel"]
    
    # Print header
    header_row = ""
    for i, header in enumerate(headers):
        header_row += f"{header:<{col_widths[i]}}"
    print(header_row)
    print("-" * sum(col_widths))
    
    # Print rows
    for i, aspect in enumerate(comparison_data["Aspect"]):
        row = f"{aspect:<{col_widths[0]}}"
        row += f"{comparison_data['LangChain'][i]:<{col_widths[1]}}"
        row += f"{comparison_data['Semantic Kernel'][i]:<{col_widths[2]}}"
        print(row)
    
    print("\\n🎯 WHEN TO CHOOSE LANGCHAIN")
    print("-" * 30)
    
    langchain_use_cases = [
        "🔗 Complex chain orchestration and workflows",
        "🛠️ Need extensive pre-built tool integrations",
        "🧠 Advanced memory and retrieval requirements",
        "🌐 Multi-provider flexibility is critical",
        "📚 Rich documentation and community support needed",
        "🔄 Rapid prototyping with diverse components",
        "🐍 Python-first development approach"
    ]
    
    for use_case in langchain_use_cases:
        print(f"   {use_case}")
    
    print("\\n🎯 WHEN TO CHOOSE SEMANTIC KERNEL")
    print("-" * 35)
    
    sk_use_cases = [
        "🏢 Enterprise Microsoft environment",
        "🚀 Quick start with minimal learning curve",
        "🔌 Plugin-based extensibility preferred",
        "⚡ Performance optimization important",
        "🛡️ Enterprise security and compliance focus",
        "🔗 Native Azure integration required",
        "🎯 Simpler, more focused agent requirements"
    ]
    
    for use_case in sk_use_cases:
        print(f"   {use_case}")
    
    print("\\n🤝 HYBRID APPROACH")
    print("-" * 17)
    print("🔄 You can use both frameworks in the same project!")
    print("   • LangChain for complex workflows and tools")
    print("   • Semantic Kernel for Microsoft-integrated components")
    print("   • Choose based on specific use case requirements")
    
    print("\\n🎓 LEARNING RECOMMENDATION")
    print("-" * 25)
    print("📚 Start with: Semantic Kernel (easier learning curve)")
    print("🔄 Then explore: LangChain (for advanced capabilities)")
    print("🎯 Choose based on: Your specific project needs")
    print("💡 Remember: Both are excellent frameworks!")
    
    return comparison_data

# Generate the comparison
comparison_results = create_framework_comparison()

print("\\n✨ Framework comparison complete!")
print("🎯 Now you can make informed decisions about which framework to use!")
print("🚀 Both workshops completed - you're ready for production AI agents!")

## Optional utilities

Use these helpers if you need to:
- Reset or clean up your environment variables and optional .env file
- Start the LangChain FastAPI server (uvicorn) from the notebook
- Stop the server safely on Windows

These are optional and independent from the Quick Start steps above. If you don't need them, you can ignore this section.

In [ ]:
# Utility: Reset config (.env and in-memory)
import os, json, shutil
from pathlib import Path

# Toggle deletion of .env file created in Step 2
DELETE_ENV_FILE = False  # set True to remove .env

# Env vars used by this LangChain app
_ENV_KEYS = [
    "AZURE_INFERENCE_ENDPOINT","AZURE_INFERENCE_CREDENTIAL","GENERIC_MODEL",
    "PROJECT_ENDPOINT","PEOPLE_AGENT_ID","KNOWLEDGE_AGENT_ID",
    "FRONTEND_URL","LOG_LEVEL","ENVIRONMENT",
    "SESSION_STORAGE_TYPE","SESSION_STORAGE_PATH","REDIS_URL",
    "DEBUG_LOGS","CONFIG_PATH"
 ]

def _mask(v):
    if v is None:
        return ""
    return v[:4] + "***" if len(v) > 8 else "***"

def reset_config(delete_env: bool = False):
    # Clear in-memory env
    cleared = {}
    for k in _ENV_KEYS:
        if k in os.environ:
            cleared[k] = os.environ.pop(k)
    # Optionally delete .env in this folder
    env_path = Path(".env")
    removed_env_file = False
    if delete_env and env_path.exists():
        try:
            env_path.unlink()
            removed_env_file = True
        except Exception as e:
            print(f"Warning: couldn't delete .env: {e}")
    print("Cleared env keys:")
    for k, v in cleared.items():
        print(f"- {k} = { _mask(v) }")
    print(f"Removed .env file: {removed_env_file}")
    return {"cleared": list(cleared.keys()), "removed_env_file": removed_env_file}

result = reset_config(DELETE_ENV_FILE)
print("Reset complete.")

In [ ]:
# Utility: Start API server (uvicorn) in background
import os, sys, subprocess, time, json
from pathlib import Path

# Settings
HOST = os.getenv("HOST", "127.0.0.1")
PORT = int(os.getenv("PORT", "8001"))  # avoid conflict with SK if it's 8000
RELOAD = False  # set True during local dev
LOG_LEVEL = os.getenv("LOG_LEVEL", "info")
PID_FILE = Path(".uvicorn_pid")

def start_server():
    if PID_FILE.exists():
        print("A server appears to be running already (PID file exists). If it's stale, run the Stop cell first.")
        return {"status": "skipped", "reason": "pid_exists"}
    cmd = [sys.executable, "-m", "uvicorn", "main:app", "--host", HOST, "--port", str(PORT), "--log-level", LOG_LEVEL]
    if RELOAD:
        cmd.append("--reload")
    # On Windows, creationflags=CREATE_NEW_PROCESS_GROUP helps Ctrl+C and termination
    creationflags = 0x00000200  # CREATE_NEW_PROCESS_GROUP
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, creationflags=creationflags)
    PID_FILE.write_text(str(proc.pid))
    print(f"Starting uvicorn main:app at http://{HOST}:{PORT} (pid={proc.pid})...")
    # Brief wait to give server time to bind
    time.sleep(1.5)
    return {"status": "started", "pid": proc.pid, "url": f"http://{HOST}:{PORT}"}

result = start_server()
result

In [ ]:
# Utility: Stop API server (read PID file and terminate)
import os, signal
from pathlib import Path

PID_FILE = Path(".uvicorn_pid")

def stop_server():
    if not PID_FILE.exists():
        print("No PID file found. If a server is running, you may need to stop it manually.")
        return {"status": "skipped", "reason": "no_pid"}
    try:
        pid = int(PID_FILE.read_text().strip())
    except Exception as e:
        print(f"Couldn't read pid: {e}")
        return {"status": "error", "error": str(e)}
    try:
        # Windows-friendly termination: first try CTRL_BREAK_EVENT, then terminate
        try:
            os.kill(pid, signal.CTRL_BREAK_EVENT)
        except Exception:
            # Fallback to terminate
            os.kill(pid, signal.SIGTERM)
        print(f"Sent termination to pid {pid}")
    except Exception as e:
        print(f"Error signaling process: {e}")
        return {"status": "error", "error": str(e)}
    try:
        PID_FILE.unlink()
    except Exception:
        pass
    return {"status": "stopped", "pid": pid}

result = stop_server()
result